In [ ]:
# First let's import the packages we will use in this project
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv(r"D:\Project 3\Supermart Grocery Sales - Retail Analytics Dataset.csv")


In [ ]:
df.info() 

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
# 1. Convert the 'Order Date' column to a proper datetime format
df['Order Date'] = pd.to_datetime(df['Order Date'], dayfirst=True, errors='coerce')

# 2. Extract specific time components into new columns
df['Year'] = df['Order Date'].dt.year
df['Month_Num'] = df['Order Date'].dt.month
df['Month_Name'] = df['Order Date'].dt.month_name()
df['Day'] = df['Order Date'].dt.day
df['Weekday'] = df['Order Date'].dt.day_name()

# 3. Create a boolean (True/False) column to identify weekends
df['Is_Weekend'] = df['Weekday'].isin(['Saturday', 'Sunday'])

In [ ]:
df.head(5)

In [ ]:
df['Sales'] = pd.to_numeric(df['Sales'],errors ='coerce')
df['Profit'] = pd.to_numeric(df['Profit'],errors ='coerce')
df['Discount'] = pd.to_numeric(df['Discount'],errors ='coerce')

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df['Profit_Margin'] = df['Profit'] / df['Sales']
df['Profit_Margin'] = df['Profit_Margin'].replace([np.inf, -np.inf], np.nan)
df['Zero_Sales_Flag'] = df['Sales'] == 0 

In [ ]:
df['Customer Name'] = df['Customer Name'].fillna('Unknown_Customer')
df['Sales'] = df['Sales'].fillna(df['Sales'].median())
df['Profit'] = df['Profit'].fillna(df['Profit'].median())
df['Discount'] = df['Discount'].fillna(0)

In [ ]:
text_columns = ['Category', 'Sub Category', 'City', 'Region', 'State']
for col in text_columns:
    df[col] = df[col].str.strip().str.title()

In [ ]:
# Identify outliers using IQR
def detect_outliers(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return lower, upper

sales_lower, sales_upper = detect_outliers(df['Sales'])
profit_lower, profit_upper = detect_outliers(df['Profit'])

df['Sales_Outlier'] = (df['Sales'] < sales_lower) | (df['Sales'] > sales_upper)
df['Profit_Outlier'] = (df['Profit'] < profit_lower) | (df['Profit'] > profit_upper)

In [ ]:
df.head()

In [ ]:
summary_stats = pd.DataFrame({
    'Total_Sales': [df['Sales'].sum()],
    'Total_Profit': [df['Profit'].sum()],
    'Avg_Order_Value': [df['Sales'].mean()],
    'Median_Sales': [df['Sales'].median()],
    'Avg_Profit_Margin': [df['Profit_Margin'].mean()],
    'Total_Orders': [len(df)]
})

In [ ]:
summary_stats

In [ ]:
plt.figure(figsize=(12, 6))
category_sales = df.groupby('Category')['Sales'].sum().sort_values(ascending=False)
sns.barplot(x=category_sales.values, y=category_sales.index)
plt.title('Total Sales by Product Category')
plt.xlabel('Total Sales')

In [ ]:
plt.figure(figsize=(10, 6))
margin_by_category = df.groupby('Category')['Profit_Margin'].mean().sort_values(ascending=False)
sns.barplot(x=margin_by_category.values, y=margin_by_category.index)
plt.title('Average Profit Margin by Category')

In [ ]:
monthly_sales = df.groupby(['Year', 'Month_Num'])['Sales'].sum().reset_index()
monthly_sales['Year-Month'] = monthly_sales['Year'].astype(str) + '-' + monthly_sales['Month_Num'].astype(str)

plt.figure(figsize=(15, 6))
plt.plot(monthly_sales['Year-Month'], monthly_sales['Sales'], marker='o')
plt.xticks(rotation=50)
plt.title('Monthly Sales Trend')


In [ ]:
# Create a cross-tabulation
category_region_sales = pd.crosstab(df['Category'], df['Region'], values=df['Sales'], aggfunc='sum')
# Plotting
category_region_sales.plot(kind='bar', stacked=True, figsize=(12, 5))
plt.title('Sales Breakdown by Category and Region')
plt.xlabel('Product Category')
plt.ylabel('Total Sales')
plt.xticks(rotation=45)
plt.legend(title='Region')
plt.tight_layout()
plt.show()

In [ ]:
df.info()

In [ ]:
plt.figure(figsize=(10, 6))

# Corrected: 'Region' is used for hue (categories) instead of 'Profit' (numbers).
sns.countplot(x="Category", hue="Region", data=df).set(
    title="Order Count by Category and Region",
    xlabel="Product Category", 
    ylabel="Count of Orders"
)

plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('category_by_region_countplot_v2.png')